# DAS Link Creation Agent - Animals Dataset

This notebook demonstrates the use of `hyperon-experimental` in conjunction with the **DAS Link Creation Agent (DAS LCA)** to generate expressions (links) through pattern-matching queries.

For full DAS documentation check [here](https://github.com/singnet/das), and more DAS LCA documentation [here](https://github.com/singnet/das/tree/master/src/agents/link_creation_agent)

**Before running this notebook, execute the setup procedure/notebook to load the [animal_dataset](https://github.com/singnet/das/blob/master/src/tests/integration/cpp/data/animal_expressions_camel.metta).**

## Setup
Install the dependencies

In [ ]:
! pip install hyperon

Import necessary modules and initialize Hyperon with DAS knowledge base

In [ ]:
import hyperon

metta = hyperon.MeTTa()
def run(program='!(+ 1 2)'):
    for result in metta.run(program):
        for child in result:
            print(child)

In [ ]:
run('!(import! &self das)')

Bind the DAS Link Creation Agent (LCA) to &das namespace

In [ ]:
run('!(bind! &das (new-das! (localhost:47000-47999) (localhost:31700)))')

## DAS LCA Configuration

Execute the command `!(das-get-params!)` to display the current DAS parameters and view the default settings.  
DAS LCA supports the following query parameters:
* context
* max_answers
* attention_update_flag
* positive_importance_flag
* use_metta_as_query_tokens

Additionally, the following parameters are specific to DAS LCA:
* repeat_count: Defines the number of creation cycles. Set to 0 to run indefinitely (default: 1)
* query_interval: Defines the interval between creation cycles (default: 0)
* query_timeout: Defines the query timeout in seconds. Set to 0 for no timeout (default: 0)


In [ ]:
run('!(das-get-params!)')
run('!(das-set-param! (max_answers 10))')

## Creating Expressions Using Templates

To generate new expressions with LCA, define a query and one or more templates. For each result produced by the query, LCA will create new expressions based on the specified templates.

For example, the query
`(and (Evaluation (Predicate "is_mammal") (Concept $C1)) (Evaluation (Predicate "is_mammal") (Concept $C2)))`
returns all pairs of animals classified as mammals.

The templates `(Similarity $C1 $C2)` and `(Similarity $C2 $C1)` will then generate two expressions for each identified pair of animals.

In [ ]:
run('''
    !(das-link-creation! 
        (and 
            (Evaluation (Predicate "is_mammal") (Concept $C1)) 
            (Evaluation (Predicate "is_mammal") (Concept $C2))
        ) 
        (Similarity $C1 $C2) 
        (Similarity $C2 $C1)
    )
    ''')

To check the expressions run:

In [ ]:
run('!(match &das (Similarity $P1 $P2) (Similarity $P1 $P2))')

Nested Templates Example:

DAS LCA supports nested expressions as templates and automatically generates all corresponding instantiated expressions.  
Given the expression `(and (Evaluation (Predicate $P1) (Concept $C1)) (Evaluation (Predicate $P1) (Concept $C2)))` and the template `(Similarity ($P1 $C1) ($P1 $C2))`,  
DAS LCA will create expressions linking the Predicate `$P1` applied to `$C1` and `$C2` through the `Similarity` relation.

Example:  
P1 = "is_animal"  
C1 = "rhino"  
C2 = "monkey"  

Result:  
`(Similarity ("is_animal" "rhino") ("is_animal" "monkey"))`

In [ ]:
run('''
    !(das-link-creation! 
        (and 
            (Evaluation (Predicate $P1) (Concept $C1)) 
            (Evaluation (Predicate $P1) (Concept $C2))
        ) 
        (Similarity ($P1 $C1) ($P1 $C2))
    )
    ''')

To check the created expressions run:

In [ ]:
run('!(match &das (Similarity ($P1 $C1) ($P1 $C2)) (Similarity ($P1 $C1) ($P1 $C2)))')

## Creating Expressions Using Processors

Instead of using templates, it is possible to create expressions with an available processor. Processors take each result produced by a query and generate new expressions according to the processor’s internal logic.

Processors are useful because they can perform **specialized computations**, such as assigning additional properties to the newly created expressions.  
For example, the **IMPLICATION_RELATION** processor calculates the implication strength between two answers and stores this value as a custom property of the generated link.  
Each processor is implemented internally within DAS LCA and encapsulates a specific algorithm or heuristic that determines how the new expressions are generated.    
New processors can be added under demand, check the current status of requests [here](https://github.com/singnet/das/issues). In future versions, DAS LCA will also support user-defined processors implemented directly in MeTTa, allowing advanced users to prototype and integrate custom processing logic without modifying the core system.

When invoking a processor, the caller must provide a query and a unique "tag" that identifies which processor should be executed for a given query.


The available processors tags are:

* IMPLICATION_RELATION

Given a query, this processor computes the strength of implication between pairs of answers and generates new expressions representing directional relationships:
If the computed strength is greater than zero, it generates new expressions and stores the strength as an additional property of the link.  
The expressions take the form:  
`(Implication A B)` and `(Implication B A)`  
which correspond to:  
A → B and B → A
* EQUIVALENCE_RELATION

Given a query, this processor computes the strength of equivalence between pairs of answers.  
If the computed strength is greater than zero, it generates expressions and stores the strength as an additional property of the link.  
The expressions take the form:  
`(Equivalence A B)` and `(Equivalence B A)`  
which correspond to:  
A ↔ B

Processor **EQUIVALENCE_RELATION** Example:

In [ ]:
run('!(das-link-creation! (and (Concept $P1) (Concept $P2)) EQUIVALENCE_RELATION)')

To check the expressions run:

In [ ]:
run('!(match &das (Equivalence $P1 $P2) (Equivalence $P1 $P2))')